In [65]:
import matplotlib.pyplot as plt
import numpy as np

from sklearnex import patch_sklearn
patch_sklearn()

from matplotlib import rc
import matplotlib.ticker as tkr
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from matplotlib.colors import ListedColormap
import seaborn as sns
sns.set_style("whitegrid")
sns.set_style("ticks")
import scipy.io as sio

import pandas as pd

import sys

%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10, 8
rcParams['figure.dpi'] = 90

%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings('ignore')

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [66]:
from sklearn import linear_model
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pylab import rc, plot
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.metrics import precision_recall_curve, classification_report
from sklearn.model_selection import train_test_split
from pandas import read_csv, DataFrame
from sklearn.metrics import roc_curve
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
# import ml_metrics, string, re, pylab as pl
from sklearn.svm import LinearSVC
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error, explained_variance_score, accuracy_score
from sklearn import svm
from sklearn.neural_network import MLPRegressor

In [70]:
dfo = pd.read_csv('/home/sir/farmy/ch.farmy.scinode/development/9631-update/farmy_bags_forecast_v2.csv')

In [76]:
dfo.bags_used.describe(percentiles=[0.01, 0.02, 0.95, 0.98, 0.99, 0.999])

count    304273.000000
mean          3.148058
std           1.932858
min           0.000000
1%            0.000000
2%            0.000000
50%           3.000000
95%           6.000000
98%           8.000000
99%           9.000000
99.9%        14.000000
max          39.000000
Name: bags_used, dtype: float64

In [77]:
dfo.total_weight.describe(percentiles=[0.01, 0.02, 0.95, 0.98, 0.99, 0.999])

count    3.045980e+05
mean     1.043594e+04
std      1.033456e+04
min      0.000000e+00
1%       0.000000e+00
2%       2.500000e+02
50%      8.258500e+03
95%      2.657500e+04
98%      3.700000e+04
99%      4.800000e+04
99.9%    8.321717e+04
max      1.012805e+06
Name: total_weight, dtype: float64

In [74]:
dfi = dfo[dfo.bags_used<10]
dfi = dfo[dfo.bags_used>0]
dfi = dfo[(dfo.total_weight<5e4)]
# dfi = dfi[(dfi.hub_id == 4)]
dfi.fillna(0.0, inplace=True)

In [75]:
dfi.deep_frozen_bags_used.describe(percentiles=[0.01, 0.02, 0.95, 0.98, 0.99, 0.999])

count    302276.000000
mean          0.186135
std           0.428129
min           0.000000
1%            0.000000
2%            0.000000
50%           0.000000
95%           1.000000
98%           1.000000
99%           2.000000
99.9%         3.000000
max           8.000000
Name: deep_frozen_bags_used, dtype: float64

In [58]:
extra_features = list() + ['day_of_week']   + ['number_of_week'] + ['delivery_hour']    # + ['day_of_year']
features = list(dfi.columns[0:-8]) + extra_features
# features

In [98]:
print(f"f65: {features[65]}")
print(f"f63: {features[63]}")
print(f"f9: {features[9]}")
print(f"f8: {features[8]}")
print(f"f61: {features[61]}")
print(f"f60: {features[60]}")
print(f"f66: {features[66]}")
print(f"f31: {features[31]}")
print(f"f30: {features[30]}")
print(f"f15: {features[15]}")

f65: number_of_week
f63: total_weight
f9: Früchte & Gemüse (cold weight)
f8: Früchte & Gemüse (cold VU)
f61: total_quantity
f60: lint_item_count
f66: delivery_hour
f31: Getränke & Weine (normal weight)
f30: Getränke & Weine (normal VU)
f15: Milch & Eier (cold weight)


In [60]:
df_enriched = dfi.copy()
df_enriched['delivery_time'] = pd.to_datetime(dfi['delivery_time'], utc=True)
df_enriched['day_of_year'] = df_enriched['delivery_time'].dt.dayofyear
df_enriched['day_of_week'] = df_enriched['delivery_time'].dt.dayofweek
df_enriched['number_of_week'] = df_enriched['delivery_time'].dt.isocalendar().week # .dt.week #
df_enriched['delivery_hour'] = df_enriched['delivery_time'].dt.hour


# 3. Filter the data for the given date range
timezone = df_enriched['delivery_time'].iloc[0].tzinfo
start_date = pd.to_datetime('2022-09-01').tz_localize(timezone)
end_date = pd.to_datetime('2023-09-01').tz_localize(timezone)
df_filtered = df_enriched[(df_enriched['delivery_time'] > start_date) & (df_enriched['delivery_time'] < end_date)]

# 4. Add 'day_of_year' and 'day_of_week' columns to the filtered data (will only execute if df_filtered is not empty)
if df_filtered.empty:
  print("filtered no data!")

# train = data.sample(frac=0.95)
# df_test = df_filtered.loc[~df_filtered.index.isin(df_filtered.index)]

sep_test = df_enriched[(df_enriched['delivery_time'] > end_date)] # | (dfi['delivery_time'] < start_date)]


In [61]:
import xgboost as xgb
from joblib import dump, load
from sklearn.model_selection import train_test_split

In [62]:
import mlflow

In [63]:
def train_model(target_column, hub_id, features):
    target_column_name = target_column+'_used'
    forecast_column_name = target_column_name+'_forecast'
    
    columns = features + [target_column_name] + [forecast_column_name]
    data = df_filtered[df_filtered.hub_id==hub_id][columns]
    data.dropna(inplace=True)
    target = [target_column_name]
    
    # train = data.sample(frac=0.95)
    # test = data.loc[~data.index.isin(train.index)]
    
    train, test = train_test_split(data, test_size=0.3, random_state=42)
    
    if (target_column == "deep_frozen_bags"):
        params = { 'bootstrap': False, 
        'ccp_alpha': 0.0, 
        'criterion': 'squared_error', 
        'max_depth': 30, 
        'max_features': 'sqrt', 
        'max_leaf_nodes': None, 
        'max_samples': None, 
        'min_impurity_decrease': 0.0, 
        'min_samples_leaf': 2, 
        'min_samples_split': 10, 
        'min_weight_fraction_leaf': 0.0, 
        'n_estimators': 800, 
        'n_jobs': 6, 
        'oob_score': False, 
        'random_state': 42, 
        'verbose': 0, 
        'warm_start': False
        }

        model = RandomForestRegressor(**params)
        
    elif (target_column == "bags"):
        params = {'colsample_bytree': 0.8049310664813739, 'gamma': 0.31904734990860323, 'learning_rate': 0.020987723995735754, 'max_depth': 8, 'n_estimators': 1400, 'objective': 'reg:squarederror', 'random_state': 42, 'subsample': 0.6878477231583816, 'verbosity': 1}
        
        model = xgb.XGBRegressor(**params, device="cuda", tree_method="gpu_hist",
                                 #objective="reg:squarederror", random_state=42, 
                                 #colsample_bytree=0.804, gamma=0.31904, 
                                 #learning_rate=0.02098, max_depth=8, n_estimators=1400, 
                                 #subsample=0.6878477231583816,
                                 #verbosity=0
                                )
        # model = mlflow.pyfunc.load_model('/home/sir/farmy/ch.farmy.scinode/development/9631-update/mlruns/8/dfeb8badb69c493d91fab39c78c9999e/artifacts/model')
    else:
        params = {'colsample_bytree': 0.9332506592696221, 'gamma': 0.33126595740822656, 'learning_rate': 0.011121117890656158, 'max_depth': 8, 'n_estimators': 700, 'objective': 'reg:linear', 'random_state': 42, 'subsample': 0.5581688088227714, 'verbosity': 1}
        model = xgb.XGBRegressor(**params, device="cuda", tree_method="gpu_hist")
        
    model.fit(train[features].values, train[target].values)
    
    
    print("Mean Squared Error between {} and:".format(target_column))
    y_forecast = data[forecast_column_name].values.flatten()
    mse_forecast = mean_squared_error(data[target].values.flatten(), y_forecast)
    print("forecast from DB:           {:.5f} (should be worst)".format(mse_forecast))

    y_test = model.predict(test[features].values).flatten()
    mse_test = mean_squared_error(test[target].values.flatten(), np.around(y_test))
    print("prediction for test slice:  {:.5f} ({:.2f}% improvement)".format(mse_test, (mse_forecast-mse_test)/mse_test*100))

    y_test = model.predict(sep_test[features].values).flatten()
    mse_sep = mean_squared_error(sep_test[target].values.flatten(), np.around(y_test))
    print("prediction for Sept slice:  {:.5f} ({:.2f}% improvement)".format(mse_sep, (mse_forecast-mse_sep)/mse_sep*100))

    y_train = model.predict(train[features].values).flatten()
    mse_train = mean_squared_error(train[target].values.flatten(), np.around(y_train))
    print("prediction for train slice: {:.5f} (should be smallest but not differ a lot from test MSE)".format(mse_train))
                
    dump(model, 'new/{}_model_v3_hub_{}.joblib'.format(target_column, hub_id), compress=True)
    return model

In [64]:
print('{}-{} for {}'.format(start_date, end_date, extra_features))
target_columns = [ 'cold_bags', 'bags'] # 'deep_frozen_bags', 
for hub_id in [1]:
    print("hub ",hub_id)
    for target_column in target_columns:
        model = train_model(target_column, hub_id, features)
print(model.get_params())

2022-09-01 00:00:00+00:00-2023-09-01 00:00:00+00:00 for ['day_of_week', 'number_of_week', 'delivery_hour']
hub  1
Mean Squared Error between cold_bags and:
forecast from DB:           0.14279 (should be worst)
prediction for test slice:  0.11760 (21.42% improvement)
prediction for Sept slice:  0.16786 (-14.94% improvement)
prediction for train slice: 0.09245 (should be smallest but not differ a lot from test MSE)


KeyboardInterrupt: 